# Big Data Distribution - Join, groupBy, Pivot
Name: Ojas Patil

Reg No. - 21BAI1106

## Problem statement:

The ABC bank customer details are given in two different csv files.

1. Perform inner join, outer join, left and right join and cross join.

2. Find the average age of the customer based on their job.

3. Consider the job and marital status of the customer and find their average age.

4. What will be the max and min duration of the customer call duration?

5. Pivot the marital status and job based on their average age

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6209118bc0bd3d2d14348b35a299ab92d52a3966cf3287392734cae9c1e07317
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('CarPricePrediction').getOrCreate()

In [4]:
spark

In [8]:
df1 = spark.read.csv('/content/ABC_1st_half.csv',header=True,inferSchema=True)
df2 = spark.read.csv('/content/ABC_2nd_half.csv',header=True,inferSchema=True)

In [9]:
df1.show()

+-------+---+-----------+--------+-------------------+-------+-------+-------+---------+
|Cust_id|age|        job| marital|          education|default|housing|   loan|  contact|
+-------+---+-----------+--------+-------------------+-------+-------+-------+---------+
|      1| 56|  housemaid| married|           basic.4y|     no|     no|     no|telephone|
|      2| 57|   services| married|        high.school|unknown|     no|     no|telephone|
|      4| 40|     admin.| married|           basic.6y|     no|     no|     no|telephone|
|      5| 56|   services| married|        high.school|     no|     no|    yes|telephone|
|      8| 41|blue-collar| married|            unknown|unknown|     no|     no|telephone|
|      9| 24| technician|  single|professional.course|     no|    yes|     no|telephone|
|     11| 41|blue-collar| married|            unknown|unknown|     no|     no|telephone|
|     12| 25|   services|  single|        high.school|     no|    yes|     no|telephone|
|     13| 29|blue-col

In [10]:
df2.show()

+-------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|Cust_id|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+-------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|      1|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      3|  may|        mon|     226|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      4|  may|        mon|     151|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      5|  may|        mon|     307|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      6|  ma

In [11]:
df1.printSchema()

root
 |-- Cust_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)



In [12]:
df2.printSchema()

root
 |-- Cust_id: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: integer (nullable = true)
 |-- y: string (nullable = true)



### 1. Perform inner join, outer join, left and right join and cross join.

In [14]:
inner_join = df1.join(df2, "Cust_id", "inner")
inner_join.show()

+-------+---+-----------+--------+-----------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|Cust_id|age|        job| marital|        education|default|housing|   loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+-------+---+-----------+--------+-----------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|      1| 56|  housemaid| married|         basic.4y|     no|     no|     no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      4| 40|     admin.| married|         basic.6y|     no|     no|     no|telephone|  may|        mon|     151|       1|  999|

In [15]:
outer_join = df1.join(df2, "Cust_id", "outer")
outer_join.show()

+-------+----+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+----+
|Cust_id| age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|   y|
+-------+----+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+----+
|      1|  56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191|  no|
|      3|NULL|       NULL|    NULL|               NULL|   NULL|   NULL|NULL|     NULL|  may|        mon|     226|       1|  

In [16]:
left_join = df1.join(df2, "Cust_id", "left")
left_join.show()

+-------+---+-----------+--------+-------------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+----+
|Cust_id|age|        job| marital|          education|default|housing|   loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|   y|
+-------+---+-----------+--------+-------------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+----+
|      1| 56|  housemaid| married|           basic.4y|     no|     no|     no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191|  no|
|      2| 57|   services| married|        high.school|unknown|     no|     no|telephone| NULL|       NULL|    NULL| 

In [17]:
right_join = df1.join(df2, "Cust_id", "right")
right_join.show()

+-------+----+-----------+-------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|Cust_id| age|        job|marital|        education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+-------+----+-----------+-------+-----------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|      1|  56|  housemaid|married|         basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      3|NULL|       NULL|   NULL|             NULL|   NULL|   NULL|NULL|     NULL|  may|        mon|     226|       1|  999|       0|nonexi

In [18]:
cross_join = df1.join(df2, "Cust_id", "cross")
cross_join.show()

+-------+---+-----------+--------+-----------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|Cust_id|age|        job| marital|        education|default|housing|   loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+-------+---+-----------+--------+-----------------+-------+-------+-------+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|      1| 56|  housemaid| married|         basic.4y|     no|     no|     no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|       5191| no|
|      4| 40|     admin.| married|         basic.6y|     no|     no|     no|telephone|  may|        mon|     151|       1|  999|

## 2. Find the average age of the customer based on their job.

In [19]:
avg_age = df1.groupBy("job").avg("age")
avg_age.show()

+-------------+------------------+
|          job|          avg(age)|
+-------------+------------------+
|   management| 42.36285909712722|
|      retired|62.031995346131474|
|      unknown| 45.53495440729483|
|self-employed|             39.95|
|      student| 25.89485714285714|
|  blue-collar| 39.55674448767834|
| entrepreneur|41.723214285714285|
|       admin.| 38.18529891565109|
|   technician|38.504227859368044|
|     services|37.928139183055976|
|    housemaid|              45.5|
|   unemployed| 39.74333662388944|
+-------------+------------------+



## 3. Consider the job and marital status of the customer and find their average age.

In [22]:
avg_age_ = df1.groupBy("job", "marital").avg("age")
avg_age_.show()

+------------+--------+------------------+
|         job| marital|          avg(age)|
+------------+--------+------------------+
|entrepreneur| married|42.527544351073765|
|     unknown| married| 48.27038626609442|
|     student| unknown|              30.0|
|  unemployed|  single|33.254980079681275|
|      admin.|  single|33.645935483870964|
| blue-collar|divorced| 42.84065934065934|
|   housemaid| married|45.745173745173744|
|     unknown|  single|36.608108108108105|
|     unknown| unknown|40.333333333333336|
|     unknown|divorced| 50.92307692307692|
|  technician|divorced| 42.05297157622739|
|entrepreneur|divorced| 44.10614525139665|
|     student|  single|25.555825242718445|
|  unemployed| unknown|              47.2|
|  unemployed| married| 41.69668246445497|
|  management| unknown|              51.0|
|     student| married|30.634146341463413|
|      admin.| married| 40.29874333587205|
|  management| married| 43.75634274772619|
|entrepreneur| unknown|              34.0|
+----------

## 4. What will be the max and min duration of the customer call duration?

In [26]:
from pyspark.sql.functions import max

max_call = df2.agg(max("duration").alias("Max call duration"))
max_call.show()

+-----------------+
|Max call duration|
+-----------------+
|             4918|
+-----------------+



In [27]:
from pyspark.sql.functions import min

min_call = df2.agg(min("duration").alias("Min call duration"))
min_call.show()

+-----------------+
|Min call duration|
+-----------------+
|                0|
+-----------------+



## 5. Pivot the marital status and job based on their average age.

In [31]:
pivot_age = df1.groupBy("marital").pivot("job").avg("age")
pivot_age.show()

+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
| marital|            admin.|       blue-collar|      entrepreneur|         housemaid|        management|           retired|    self-employed|         services|           student|       technician|        unemployed|           unknown|
+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
| unknown|35.785714285714285| 41.57142857142857|              34.0|              40.0|              51.0|              61.0|             39.4|             40.0|              30.0|            32.75|              47.2|40.333333333333336|
|divorced|       43.28203125| 42.84065934065934| 44.1061